In [105]:
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET 
import pprint
import re
import csv
import codecs
import cerberus
import schema
from collections import defaultdict

SAMPLE_FILE = "sample.osm"
OSM_FILE = "boston_massachusetts.osm"
SCHEMA = schema.schema

LOWER = re.compile(r'^([a-z]|_)*$')
LOWER_COLON = re.compile(r'^([a-z]|_)*:([a-z]|_)*')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


### 生成样本文件   

原始数据```boston_massachusetts.osm```大小为418M。   
当k取值45，获得样本```sample.osm```，大小为9.39M，符合样本大小要求。 
 

In [4]:
k = 45 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)  
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem  # 包含yield语句,编译成生成器,生成迭代值后挂起并暂停，下次继续运行
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):  #将可遍历对象组合为索引序列
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

### 解析文件,查看标记种类和数量

In [7]:
def count_tags(filename):    
    tag_data = {}   # 计数存入字典中
    for event, elem in ET.iterparse(filename, events=('start',)):
        if elem.tag in tag_data:
            tag_data[elem.tag] += 1   # 已包含该tag，计数+1
        else:
            tag_data[elem.tag] = 1    # 未包含该tag，新建键值对
        
        elem.clear()
    return tag_data

print 'sample:'
print count_tags(SAMPLE_FILE)

print 'osm_file:'
print count_tags(OSM_FILE)



sample:
{'node': 43398, 'nd': 51839, 'member': 278, 'tag': 20202, 'relation': 30, 'way': 6912, 'osm': 1}
osm_file:
{'node': 1952868, 'nd': 2350810, 'bounds': 1, 'member': 11275, 'tag': 911457, 'osm': 1, 'way': 311064, 'relation': 1325}


### 探索有多少用户作出了贡献

In [8]:
# 提取'uid'属性值
def get_user(element): 
    if 'uid' in element.attrib:
        return element.attrib['uid']

# 使用set()获取不重复的‘uid’
def get_unique_user(filename):
    users = set() 
    for _, element in ET.iterparse(filename):  
        if get_user(element):
            users.add(get_user(element)) 
    return users

unique_users = get_unique_user(OSM_FILE)
print 'count of unique users:', len(unique_users)

count of unique users: 1439


### 检查‘tag'标记类型

In [6]:

def key_type(element, keys): 
    if element.tag == 'tag':
        for tag in element.iter('tag'):
            if LOWER.search(tag.attrib['k']):
                keys['lower'] += 1
            elif LOWER_COLON.search(tag.attrib['k']):
                keys['lower_colon'] += 1
            elif PROBLEMCHARS.search(tag.attrib['k']):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1 
    return keys

def check_key_type(filename):
    keys = {'lower': 0, 'lower_colon': 0, 'problemchars': 0, 'other': 0}
    open_file = open(filename, 'r')
    for _, element in ET.iterparse(open_file,events=('start', )):
        keys = key_type(element, keys)
    open_file.close()
    return keys

#sample_keys = check_key_type(SAMPLE_FILE)
#print sample_keys
osm_keys = check_key_type(OSM_FILE)
print 'osm_keys', osm_keys

osm_keys {'problemchars': 3, 'lower': 796417, 'other': 478, 'lower_colon': 114559}


### 检查'tag'标记的属性

In [37]:
def check_type_keys(filename, type_re):
    type_keys = {}
    open_file = open(filename, 'r')
    for _, element in ET.iterparse(open_file,events=('start', )):
        if element.tag == 'tag':
            for tag in element.iter('tag'):
                if type_re.search(tag.attrib['k']):
                    if tag.attrib['k'] in type_keys:
                        type_keys[tag.attrib['k']] += 1
                    else:
                        type_keys[tag.attrib['k']] = 1
    open_file.close()
    return type_keys


#### 对含有冒号":"的key分类计数

In [44]:
# 含有冒号":"的key
osm_lower_colon_keys = check_type_keys(OSM_FILE, LOWER_COLON)
osm_lower_colon_keys

{'abandoned:railway': 2,
 'access:conditional': 1,
 'access:employee': 51,
 'addr:city': 3436,
 'addr:city_parent': 7,
 'addr:country': 133,
 'addr:county': 7,
 'addr:description': 1,
 'addr:floor': 1,
 'addr:full': 2,
 'addr:housename': 149,
 'addr:housenumber': 7894,
 'addr:inclusion': 35,
 'addr:interpolation': 47,
 'addr:neighbourhood': 6,
 'addr:place': 5,
 'addr:postcode': 2760,
 'addr:state': 2513,
 'addr:street': 6545,
 'addr:street_1': 1,
 'addr:unit': 9,
 'aerodrome:type': 1,
 'alt_name:ru': 1,
 'alt_name:vi': 1,
 'amenity:1997-2015': 1,
 'area:highway': 52,
 'bicycle:oneway': 1,
 'bridge:material': 1,
 'bridge:movable': 2,
 'bridge:name': 4,
 'bridge:structure': 3,
 'bridge:structure:beam': 1,
 'bridge:support': 11,
 'building:architecture': 2,
 'building:colour': 627,
 'building:height': 419,
 'building:levels': 3097,
 'building:material': 1025,
 'building:min_level': 124,
 'building:name': 1,
 'building:part': 1711,
 'building:roof:colour': 3,
 'building:roof:shape': 4,
 '

#### 对异常标签的key分类计数

In [45]:
# 查看匹配为problemchars的key
osm_problemchars_keys = check_type_keys(OSM_FILE, PROBLEMCHARS)
osm_problemchars_keys

{'(new tag)': 2, 'service area': 1}

#### 提取感兴趣标签的key及value

In [69]:
# 从文件中获取目标key及value
def get_key_value(filename, target_keys):
    all_targets = defaultdict(set)
    open_file = open(filename, 'r')
    for _, element in ET.iterparse(open_file,events=('start', )):
        if element.tag == 'tag':
            for tag in element.iter('tag'):
                if tag.attrib['k'] in target_keys:
                    all_targets[tag.attrib['k']].add(tag.attrib['v']) 
                    
    open_file.close()
    return all_targets

In [71]:
# 从源文件中提取所有感兴趣的key，再分别查看是否有问题
target_keys = ['addr:postcode', 'addr:state', 'addr:street', 'addr:street_1',
               'addr:unit', '(new tag)', 'service area']
all_target_keys = get_key_value(OSM_FILE, target_keys)

defaultdict(<type 'set'>, {'addr:unit': set(['A', 'Suite 8', '1425', '610', 'BO4', '3', '2', '1400']), 'addr:postcode': set(['02186', '02184', '02134-1327', '02189', '02134-1322', '02134-1321', '02138-1901', '02132-3226', '01821', '02134-1433', '02108', '02109', '02476', '02474', '02472', '02139', '02134-1319', 'MA 02135', '02478', '02136-2460', '02131-3025', '02136', '02140-1340', '02134', '02205', '02132', '02131', '02130', '01854', '02110-1301', '02138', '02138-2903', '02138-2901', '02134-1442', '01250', '02132-1239', '02445-7638', '02446', '02445', '02138-2742', '02120', '02121', '02210', '02124', '02125', '02126', '02215', '02128', '02129', '02474-8735', '01240', '20052', '02114-3203', '02458', '02459', 'MA 02118', 'MA 02116', '01125', '02026', 'MA 02186', '02155', 'MA', '02150', '02152', '02159', '02026-5036', '01238', '02138-2724', '02445-5841', '02138-2933', '02144', '02145', '02142', '02143', '02140', '02141', '02148', '02149', '02138-2735', '02138-2736', '02131-4931', '02118'

#### 依次查看感兴趣的key和value

In [75]:
print 'service area:', all_target_keys['service area']

service area: set(['20 miles'])


===> key值```service area```可修正为```service_area```。

In [76]:
print '(new tag):', all_target_keys['(new tag)']

(new tag): set(['Pine Avenue'])


===> key值```(new tag)```可修正为```new_tag```。

In [77]:
print 'addr:unit:', all_target_keys['addr:unit']

addr:unit: set(['A', 'Suite 8', '1425', '610', 'BO4', '3', '2', '1400'])


===> key值```addr:unit```无需修正。

In [78]:
print 'addr:street_1:', all_target_keys['addr:street_1']

addr:street_1: set(['Railroad Street'])


===> key值```addr:street_1```可修正为```addr:street```。

In [80]:
print 'postcode'
pprint.pprint(all_target_keys['addr:postcode'])

postcode
set(['01125',
     '01238',
     '01240',
     '01250',
     '01754',
     '01821',
     '01854',
     '01944',
     '02026',
     '02026-5036',
     '02043',
     '02108',
     '02109',
     '02110',
     '02110-1301',
     '02111',
     '02113',
     '02114',
     '02114-3203',
     '02115',
     '02116',
     '02118',
     '02119',
     '02120',
     '02121',
     '02122',
     '02124',
     '02125',
     '02126',
     '02127',
     '02128',
     '02129',
     '02130',
     '02130-4803',
     '02131',
     '02131-3025',
     '02131-4931',
     '02132',
     '02132-1239',
     '02132-3226',
     '02134',
     '02134-1305',
     '02134-1306',
     '02134-1307',
     '02134-1311',
     '02134-1312',
     '02134-1313',
     '02134-1316',
     '02134-1317',
     '02134-1318',
     '02134-1319',
     '02134-1321',
     '02134-1322',
     '02134-1327',
     '02134-1409',
     '02134-1420',
     '02134-1433',
     '02134-1442',
     '02135',
     '02136',
     '02136-2460',
     '0

===> key值```addr:postcode```无需修正。

In [88]:
print 'addr:state:', all_target_keys['addr:state']

addr:state: set(['ma', 'WA', 'MA- MASSACHUSETTS', 'MASSACHUSETTS', 'Ma', 'MA', 'Massachusetts'])


===> key值```addr:state```的value值需要进行修正。

In [87]:
# 检查addr:street的值规范问题
streets = all_target_keys['addr:street']
street_entry = defaultdict(list)
for street in streets:
    street_end = street.split(" ")[-1]
    street_entry[street_end].append(street)
street_entry

defaultdict(list,
            {'': ['Dartmouth ', 'SYCAMORE ST '],
             '#12': ['Harvard St #12'],
             '#1302': ['Cambridge Street #1302'],
             '#501': ['Bromfield Street #501'],
             '104': ['Mill Street, Suite 104'],
             '1100': ['First Street, Suite 1100'],
             '1702': ['Franklin Street, Suite 1702'],
             '3': ['Kendall Square - 3'],
             '303': ['First Street, Suite 303'],
             '6': ['South Station, near Track 6'],
             '846028': ['PO Box 846028'],
             'Albany': ['Albany'],
             'Artery': ['Southern Artery'],
             'Ave': ['Highland Ave',
              'Morrison Ave',
              'Francesca Ave',
              'Mystic Ave',
              'Commonwealth Ave',
              'Lexington Ave',
              'Willow Ave',
              'Washington Ave',
              'Columbus Ave',
              'Harrison Ave',
              'Somerville Ave',
              'Western Ave',
       

===> key值```addr:street```的value值需要进行修正。

#### ‘tag'标记属性修正方案
+ 按```key_mapping```修正key值   
+ 分别按```state_mapping```、 ```street_mapping```修正"addr:state'和'addr:street'
    
  
    

In [92]:
key_mapping = {"(new tag)": "new_tag",
               "service area": "service_area",
               "addr:street_1": "addr:street"}

state_mapping = {'ma': 'MA',
                 'MA- MASSACHUSETTS' : 'MA',
                 'Ma' : 'MA',
                 'Massachusetts' : 'MA'}

street_mapping = { "St": "Street",
                    "St.": "Street",
                    "ST" : "Street",
                    "St" : "Street",
                    "St," : "Street",
                    "st" : "Street",
                    "street": "Street",
                    "Street." : "Street",
                    "Rd" : "Road",
                    "Rd.": "Road",
                    "rd.": "Road",
                    "Ave": "Avenue",
                    "Ave.":"Avenue",
                    "HIghway": "Highway",
                    "Pkwy" : "Parkway",
                    "Sq." : "Square",
                    "Winsor": "Winsor",
                     }

### 将数据修正后写入CSV文件

In [114]:
# 修正key值
def correct_key(key):
    if key in key_mapping.keys():
        key = key_mapping[key]
    return key  

# 修正value值
def correct_value(key, value):
    
    if key == 'addr:state':
        if value in state_mapping.keys():
            value = state_mapping[value]
            
    if key == 'addr:street':
        abbr = value.split(" ")[-1] 
        if abbr in street_mapping.keys():
            value = value.replace(abbr, street_mapping[abbr]) 
            
    return value

# 修正type值    
def shape_tag(key):
    if LOWER_COLON.search(key):
        colon_pos = key.index(":")
        k = key[colon_pos+1 :]
        t = key[: colon_pos]
    else:
        k = key
        t = 'regular'
    return k, t   
                
# 修正根元素root    
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        
        # "node"
        for tag in node_attr_fields:  
            node_attribs[tag] = element.attrib[tag]        
        
        for subelem in element:
            
            # "node_tags"
            if subelem.tag == 'tag':
                
                k = correct_key(subelem.attrib['k'])
                v = correct_value(k, subelem.attrib['v'])
                
                entry = {}
                entry['id'] = element.attrib['id']
                entry['key'], entry['type'] = shape_tag(k)              
                entry['value'] = v
                
                tags.append(entry)
       
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        
        # "way"
        for tag in way_attr_fields:
            way_attribs[tag] = element.attrib[tag] 
            
        count = 0    
        for subelem in element:
            
            # "way_tags"
            if subelem.tag == 'tag':
              
                k = correct_key(subelem.attrib['k'])
                v = correct_value(k, subelem.attrib['v'])
                
                entry = {}
                entry['id'] = element.attrib['id']
                entry['key'], entry['type'] = shape_tag(k)              
                entry['value'] = v
                
                tags.append(entry)
            
            # "way_nodes"
            if subelem.tag == 'nd':
                entry = {}
                entry['id'] = element.attrib['id']
                entry['node_id'] = subelem.attrib['ref']
                entry['position'] = count
                count += 1 
                way_nodes.append(entry)           
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}    

In [107]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
# 依次获取根元素
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
       

 # ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

# Note: Validation is ~ 10X slower. For the project consider using a small
# sample of the map when validating.

In [115]:
# 处理sample文件并验证
NODES_PATH = "sample_nodes.csv"
NODE_TAGS_PATH = "sample_nodes_tags.csv"
WAYS_PATH = "sample_ways.csv"
WAY_NODES_PATH = "sample_ways_nodes.csv"
WAY_TAGS_PATH = "sample_ways_tags.csv"

process_map(SAMPLE_FILE, validate=True)

In [118]:
# 处理osm文件
NODES_PATH = "csv/nodes.csv"
NODE_TAGS_PATH = "csv/nodes_tags.csv"
WAYS_PATH = "csv/ways.csv"
WAY_NODES_PATH = "csv/ways_nodes.csv"
WAY_TAGS_PATH = "csv/ways_tags.csv"

process_map(OSM_FILE, validate=False)

### 将CSV文件导入数据库中



创建数据库，修改分隔符。
```sql
PS ***> sqlite3 BostonOSM.db
SQLite version 3.9.2 2015-11-02 18:31:45
Enter ".help" for usage hints.
sqlite> .show
        echo: off
         eqp: off
  explain: off
     headers: off
        mode: list
   nullvalue: ""
      output: stdout
colseparator: "|"
rowseparator: "\n"
       stats: off
       width:
sqlite> .separator ","
sqlite> .show
        echo: off
         eqp: off
  explain: off
     headers: off
        mode: list
   nullvalue: ""
      output: stdout
colseparator: ","
rowseparator: "\n"
       stats: off
       width:
```
创建表格并查看：
```sql
sqlite> CREATE TABLE nodes (
   ...>     id INTEGER PRIMARY KEY NOT NULL,
   ...>     lat REAL,
   ...>     lon REAL,
   ...>     user TEXT,
   ...>     uid INTEGER,
   ...>     version INTEGER,
   ...>     changeset INTEGER,
   ...>     timestamp TEXT
   ...> );
sqlite>
sqlite> CREATE TABLE nodes_tags (
   ...>     id INTEGER,
   ...>     key TEXT,
   ...>     value TEXT,
   ...>     type TEXT,
   ...>     FOREIGN KEY (id) REFERENCES nodes(id)
   ...> );
sqlite>
sqlite> CREATE TABLE ways (
   ...>     id INTEGER PRIMARY KEY NOT NULL,
   ...>     user TEXT,
   ...>     uid INTEGER,
   ...>     version TEXT,
   ...>     changeset INTEGER,
   ...>     timestamp TEXT
   ...> );
sqlite>
sqlite> CREATE TABLE ways_tags (
   ...>     id INTEGER NOT NULL,
   ...>     key TEXT NOT NULL,
   ...>     value TEXT NOT NULL,
   ...>     type TEXT,
   ...>     FOREIGN KEY (id) REFERENCES ways(id)
   ...> );
sqlite>
sqlite> CREATE TABLE ways_nodes (
   ...>     id INTEGER NOT NULL,
   ...>     node_id INTEGER NOT NULL,
   ...>     position INTEGER NOT NULL,
   ...>     FOREIGN KEY (id) REFERENCES ways(id),
   ...>     FOREIGN KEY (node_id) REFERENCES nodes(id)
   ...> );
sqlite> .tables
nodes       nodes_tags  ways        ways_nodes  ways_tags
```
导入csv文件，出现错误```INSERT failed: datatype mismatch```，我查看了nodes前5行数据，并没发现异常之处。参考论坛帖子发现第一行mismatch应该是由于csv header全为字符串，与schema不符导致报错，不会影响数据导入和查询。
```sql
sqlite> .import nodes.csv nodes
nodes.csv:1: INSERT failed: datatype mismatch
sqlite> SELECT * FROM nodes LIMIT 5;
30730952,42.3678097,-71.0218711,wambag,326503,2,14335103,2012-12-19T19:24:31Z
30730953,42.3677364,-71.0218568,wambag,326503,2,14335103,2012-12-19T19:24:31Z
30730954,42.3676084,-71.0218168,wambag,326503,2,14335103,2012-12-19T19:24:31Z
30730955,42.3675229,-71.0218486,wambag,326503,2,14335103,2012-12-19T19:24:32Z
30730956,42.3674548,-71.0218865,wambag,326503,2,14335103,2012-12-19T19:24:31Z
sqlite> .import nodes_tags.csv nodes_tags
sqlite> .import ways.csv ways
ways.csv:1: INSERT failed: datatype mismatch
sqlite> .import ways_tags.csv ways_tags
sqlite> .import ways_nodes.csv ways_nodes
```